# Points

## Day 1 - 30 Day Map Challenge

For this map I wanted to demonstrate how to use the [Wherobots Open Data Catalog](https://docs.wherobots.services/1.2.0/tutorials/opendata/introduction/#open-data-catalog) with [SedonaDB](https://wherobots.com/sedona-db/) to use the [Overture Maps places dataset](https://medium.com/@dr.jiayu/harnessing-overture-maps-data-apache-sedonas-journey-from-parquet-to-geoparquet-d99f7767a499). Since Day 1 is the day after Halloween we'll find dentists nearby!


![](../img/sedonadb.png)

You can follow along by creating a free account on [Wherobots Cloud.](https://www.wherobots.services)

In [ ]:
from sedona.spark import *
import os
import time

In [ ]:
config = SedonaContext.builder(). \
    config("spark.sql.catalog.wherobots_examples.type", "hadoop"). \
    config("spark.sql.catalog.wherobots_examples", "org.apache.iceberg.spark.SparkCatalog"). \
    config("spark.sql.catalog.wherobots_examples.warehouse", "s3://wherobots-examples-prod/havasu/warehouse"). \
    config("spark.sql.catalog.wherobots_examples.io-impl", "org.apache.iceberg.aws.s3.S3FileIO"). \
    getOrCreate()

sedona = SedonaContext.create(config)

In [ ]:
# Load Overture Maps Places and Admins themes as SedonaDB tables from Wherobots Data Catalog

sedona.table("wherobots_examples.overture.places_place").createOrReplaceTempView("places")
sedona.table("wherobots_examples.overture.admins_locality").createOrReplaceTempView("admins")

Learn more about Wherobots open data in the [Wherobots documentation.](https://docs.wherobots.services/1.2.0/tutorials/opendata/introduction/)

In [ ]:
# Query SedonaDB to find all dentists in Colorado

dentist_df = sedona.sql("""
SELECT places.geometry, 
    places.categories.main AS category,
    places.names.common[0].value AS name,
    places.addresses[0].freeform AS address,
    places.addresses[0].locality AS city
FROM places, admins
WHERE ( 
    ST_Contains(admins.geometry, places.geometry) 
        AND places.categories.main = 'dentist' 
        AND admins.names.common[0].value = 'Colorado'
    )
""")

dentist_df.show(5)

```
+--------------------+--------+--------------------+------------------+-----------+
|            geometry|category|                name|           address|       city|
+--------------------+--------+--------------------+------------------+-----------+
|POINT (-104.77973...| dentist|Marr Family Denti...|     1813 61st Ave|    Greeley|
|POINT (-104.94772...| dentist|Emergency Dental ...|    4200 E 8th Ave|     Denver|
|POINT (-105.07161...| dentist|Elevated Dental Arts|     418 8th St SE|   Loveland|
|POINT (-104.82244...| dentist|Platteville Famil...|    340 Justin Ave|Platteville|
|POINT (-105.02548...| dentist| ISmile Dental Group|677 S Federal Blvd|     Denver|
+--------------------+--------+--------------------+------------------+-----------+
only showing top 5 rows
```

In [ ]:
SedonaKepler.create_map(dentist_df, "Dentists In Colorado")

![](../img/01-points-dentists.png)